#Neighborhoods of Toronto
###Part 1: Importing data Wikipedia <br>Applied Data Science Capstone(Week 3 Project)
<p>Objective this part of the assignment is to scrap data from given wikipedia page to analyze <B>Neighborhoods of Toronto</B>

####Beutiful soup librery is used below to scrap data from wikipedia page.



In [0]:
#import related libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import breautiful soup to scrap data from websites 
from bs4 import BeautifulSoup

##Fetch the wikipedia page and parsed using Beautiful Soup to scrap data

In [0]:
#Below codes are used to fetch the data from wikipedia by using Request method
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(URL).text
# parse the link using Beautiful soup to get stracture dat 
soup = BeautifulSoup(res,'lxml')


##Preparing a two dimentional list to collect only required data

In [0]:
#Initiating Location List. This list will be used to gater all the respecive data
location_list = [['PostCode', 'Borough', 'Neighborhood']]

#A temp file is created to keep all unique PostCodes
postcode_temp = []

#PostCode dedup function is created to identify the index of existing Postcode in the location list. 
def postcode_dedup(data, search):
    for i, e in enumerate(data):
        try:
            return i, e.index(search)
        except ValueError: pass

#Below "for loop" is used to travers through the table data  
for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        #get post postcode, Borough and Neighbourhood from the soup file 
        Postcode = data[0].text.strip()
        Borough = data[1].text.strip()
        Neighbourhood = data[2].text.strip()
    except IndexError:pass
    
    #If Post code was found for the first time we will procceed appending the list with Postcode, Borough and Neighbourhood data
    if Postcode not in postcode_temp: 
      postcode_temp.append(Postcode) #appending ptcode data to the temporary postcode list

      #check if Brough has legitimate values, otherwise skip the entry
      if Borough != "Not assigned":
        
        #if Neighbourhood data is missing we will replace the missing data wil the Borough data 
        if Neighbourhood == "Not assigned":
          Neighbourhood = Borough
        location_list.append([Postcode, Borough, Neighbourhood])
    else:
      #finding the index to post code if its already exists and updating adding neighbourhood data with the existing neighbourhood data of that postcode
       key = postcode_dedup(location_list, Postcode)
       location_list[key[0]][2] = location_list[key[0]][2] + ', ' + Neighbourhood


##Transforming List to Pandas Dataframe

In [30]:
#importing and prparing Pandas dataset
df = pd.DataFrame(location_list, columns =['Postcode', 'Borough', 'Neighbourhood']) 
df = df[1:]

pd.set_option('display.max_colwidth', -1)
df.head(10)

,Postcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Queen's Park,Queen's Park
6,M9A,Etobicoke,Islington Avenue
7,M1B,Scarborough,"Rouge, Malvern"
8,M3B,North York,Don Mills North
9,M4B,East York,"Woodbine Gardens, Parkview Hill"
10,M5B,Downtown Toronto,"Ryerson, Garden District"


In [31]:
print("The shape of dataset is: "+str(df.shape))

The shape of dataset is: (103, 3)
